In [1]:
import pandas as pd
import numpy as np
import math
from pathlib import Path

from db_psql_model import DatabaseCursor

pd.set_option("display.max_rows", 9999)
pd.set_option("display.max_colwidth", 40)
pd.set_option("display.max_columns", 999)
pd.set_option("display.precision", 2)


PATH = list(Path().cwd().parent.glob("**/private.yaml"))[0]
OPTION_DEV = "-c search_path=dev"
OPTION_PROD = "-c search_path=prod"
# GAME_ID = 406  #4 Team 2021
GAME_ID = 359  # 6 Team 2016
GAME_ID = 273 #8 team 2012


league_settings_query = f"SELECT * from dev.leaguesettings where game_id = {GAME_ID}"
league_settings = DatabaseCursor(PATH, options=OPTION_DEV)\
    .copy_data_from_postgres(league_settings_query)
    
num_playoff_teams = league_settings["num_playoff_teams"].values[0]

playoff_start_week = league_settings["playoff_start_week"].values[0]

endofseason_rankings_query = f"SELECT game_id, reg_season_rank, manager, team_name, team_key FROM prod.regseasonrankings where game_id = {GAME_ID}"
eos_rankings = DatabaseCursor(PATH, options=OPTION_PROD)\
    .copy_data_from_postgres(endofseason_rankings_query)

team_points_weekly_query = f"SELECT * from dev.weeklyteampoints where game_id = {GAME_ID} and week >= {playoff_start_week}"
team_points_weekly = DatabaseCursor(PATH, options=OPTION_DEV)\
    .copy_data_from_postgres(team_points_weekly_query)

playoff_teams = list(
    eos_rankings["team_key"][eos_rankings["reg_season_rank"] <= num_playoff_teams]
    )

team_points_weekly["team_key"] = (
    team_points_weekly["game_id"].astype(str)
    + ".l."
    + team_points_weekly["league_id"].astype(str)
    + ".t."
    + team_points_weekly["team_id"].astype(str)
)

team_points_weekly = team_points_weekly[team_points_weekly["team_key"].isin(playoff_teams)]
team_points_weekly.sort_values(["week", "team_id"])

Successfully pulled: SELECT * from dev.leaguesettings where game_id = 273
Successfully pulled: SELECT game_id, reg_season_rank, manager, team_name, team_key FROM prod.regseasonrankings where game_id = 273
Successfully pulled: SELECT * from dev.weeklyteampoints where game_id = 273 and week >= 15


,final_points,week,projected_points,team_id,game_id,league_id,team_key
0,65.86,15,102.73,1,273,777818,273.l.777818.t.1
14,71.90,15,58.54,2,273,777818,273.l.777818.t.2
3,75.38,15,85.51,4,273,777818,273.l.777818.t.4
5,77.14,15,90.89,6,273,777818,273.l.777818.t.6
6,88.50,15,106.62,7,273,777818,273.l.777818.t.7
16,63.68,15,110.55,9,273,777818,273.l.777818.t.9
17,106.34,15,117.06,10,273,777818,273.l.777818.t.10
7,135.44,16,96.65,1,273,777818,273.l.777818.t.1
18,69.02,16,71.45,2,273,777818,273.l.777818.t.2
10,120.76,16,93.67,4,273,777818,273.l.777818.t.4


In [2]:
def printMatches(matches):
    print("Active Matches:")
    for match in matches:
        if match.is_ready_to_start():
            print(
                "\t{} vs {}".format(
                    *[p.get_competitor() for p in match.get_participants()]
                )
            )


def add_win(tourney, competitor):
    m = tourney.get_active_matches_for_competitor(competitor)[0]
    tourney.add_win(m, competitor)


def checkActiveMatches(tourney, competitorPairs):
    matches = tourney.get_active_matches()
    if len(competitorPairs) != len(matches):
        printMatches(matches)
        print(competitorPairs)
        raise Exception(
            "Invalid number of competitors: {} vs {}".format(
                len(matches), len(competitorPairs)
            )
        )
    for match in matches:
        inMatches = False
        for competitorPair in competitorPairs:
            participants = match.get_participants()
            if competitorPair[0] == participants[0].get_competitor():
                if competitorPair[1] == participants[1].get_competitor():
                    inMatches = True
            elif competitorPair[0] == participants[1].get_competitor():
                if competitorPair[1] == participants[0].get_competitor():
                    inMatches = True
    if not inMatches:
        printMatches(matches)
        print(competitorPairs)
        # raise Exception("Wrong matches")

In [3]:
class Participant:
    """
    The Participant class represents a participant in a specific match.
    It can be used as a placeholder until the participant is decided.
    """

    def __init__(self, competitor=None):
        self.competitor = competitor

    def get_competitor(self):
        """
        Return the competitor that was set,
        or None if it hasn't been decided yet
        """
        return self.competitor

    def set_competitor(self, competitor):
        """
        Set competitor after you've decided who it will be,
        after a previous match is completed.
        """
        self.competitor = competitor


class Match:
    """
    A match represents a single match in a tournament, between 2 participants.
    It adds empty participants as placeholders for the winner and loser,
    so they can be accessed as individual object pointers.
    """

    def __init__(self, left_participant, right_participant):
        self.__left_participant = left_participant
        self.__right_participant = right_participant
        self.__winner = Participant()
        self.__loser = Participant()

    def set_winner(self, competitor):
        """
        When the match is over, set the winner competitor here and the loser will be set too.
        """
        if competitor == self.__left_participant.get_competitor():
            self.__winner.set_competitor(competitor)
            self.__loser.set_competitor(self.__right_participant.get_competitor())
        elif competitor == self.__right_participant.get_competitor():
            self.__winner.set_competitor(competitor)
            self.__loser.set_competitor(self.__left_participant.get_competitor())
        else:
            raise Exception("Invalid competitor")

    def get_winner_participant(self):
        """
        If the winner is set, get it here. Otherwise this return None.
        """
        return self.__winner

    def get_loser_participant(self):
        """
        If the winner is set, you can get the loser here. Otherwise this return None.
        """
        return self.__loser

    def get_participants(self):
        """
        Get the left and right participants in a list.
        """
        return [self.__left_participant, self.__right_participant]

    def is_ready_to_start(self):
        """
        This returns True if both of the participants coming in have their competitors "resolved".
        This means that the match that the participant is coming from is finished.
        It also ensure that the winner hasn't been set yet.
        """
        is_left_resolved = self.__left_participant.get_competitor() is not None
        is_right_resolved = self.__right_participant.get_competitor() is not None
        is_winner_resolved = self.__winner.get_competitor() is not None
        return is_left_resolved and is_right_resolved and not is_winner_resolved


class Tournament:
    """
    This is a single-elimination tournament where each match is between 2 competitors.
    It takes in a list of competitors, which can be strings or any type of Python object,
    but they should be unique. They should be ordered by a seed, with the first entry being the most
    skilled and the last being the least. They can also be randomized before creating the instance.
    Optional options dict fields:
    """

    def __init__(self, competitors_list, options={}):
        assert len(competitors_list) > 1
        self.__matches = []
        next_higher_power_of_two = int(
            math.pow(2, math.ceil(math.log2(len(competitors_list))))
        )
        winners_number_of_byes = next_higher_power_of_two - len(competitors_list)
        incoming_participants = list(map(Participant, competitors_list))
        incoming_participants.extend([None] * winners_number_of_byes)

        while len(incoming_participants) > 1:
            half_length = int(len(incoming_participants) / 2)
            first = incoming_participants[0:half_length]
            last = incoming_participants[half_length:]
            last.reverse()
            next_round_participants = []
            for participant_pair in zip(first, last):
                if participant_pair[1] is None:
                    next_round_participants.append(participant_pair[0])
                elif participant_pair[0] is None:
                    next_round_participants.append(participant_pair[1])
                else:
                    match = Match(participant_pair[0], participant_pair[1])
                    next_round_participants.append(match.get_winner_participant())
                    self.__matches.append(match)
            incoming_participants = next_round_participants
        self.__winner = incoming_participants[0]

    def __iter__(self):
        return iter(self.__matches)

    def get_active_matches(self):
        """
        Returns a list of all matches that are ready to be played.
        """
        return [match for match in self.get_matches() if match.is_ready_to_start()]

    def get_matches(self):
        """
        Returns a list of all matches for the tournament.
        """
        return self.__matches

    def get_active_matches_for_competitor(self, competitor):
        """
        Given the string or object of the competitor that was supplied
        when creating the tournament instance,
        returns a list of Matches that they are currently playing in.
        """
        matches = []
        for match in self.get_active_matches():
            competitors = [
                participant.get_competitor() for participant in match.get_participants()
            ]
            if competitor in competitors:
                matches.append(match)
        return matches

    def get_winners(self):
        """
        Returns None if the winner has not been decided yet,
        and returns a list containing the single victor otherwise.
        """
        if len(self.get_active_matches()) > 0:
            return None
        return [self.__winner.get_competitor()]

    def add_win(self, match, competitor):
        """
        Set the victor of a match, given the competitor string/object and match.
        """
        match.set_winner(competitor)

In [5]:
# round_1 = [[1, 8], [2, 7], [3, 6], [4, 5]]
# round_2 = [[18, 45], [27, 36]]
# round_3 = [[1845, 2736]]
# champion = []
# tourney = Tournament(playoff_teams)
# checkActiveMatches(tourney, round_1)
# add_win(tourney, '273.l.777818.t.7')
# add_win(tourney, '273.l.777818.t.12')
# add_win(tourney, '273.l.777818.t.4')
# add_win(tourney, '273.l.777818.t.6')
# checkActiveMatches(tourney, round_2)
# add_win(tourney, '273.l.777818.t.6')
# add_win(tourney, '273.l.777818.t.4')
# checkActiveMatches(tourney, round_3)

In [2]:
next_higher_power_of_two = int(
    math.pow(
        2, 
        math.ceil(
            math.log2(len(playoff_teams))
            )
        )
    )

winners_number_of_byes = next_higher_power_of_two - len(playoff_teams)

incoming_participants = list(playoff_teams)

incoming_participants.extend(["Bye"] * winners_number_of_byes)

num_of_rounds = int(
    math.ceil(
        math.log2(len(incoming_participants))
        )
    )

num_of_matches = int(
    (len(incoming_participants) / 2) * num_of_rounds
    )

matches_per_round = int(
    num_of_matches / num_of_rounds
    )

half_length = int(
    len(incoming_participants) / 2
    )

first = incoming_participants[0:half_length]

last = incoming_participants[half_length:]
last.reverse()

round_1_matchups = list(zip(first, last))

round_1 = team_points_weekly[
    [
        "week", 
        "team_key", 
        "final_points", 
        "projected_points"
        ]
    ][
        (team_points_weekly["week"] == playoff_start_week)
        ]

round_1 = round_1.merge(
    eos_rankings, 
    how="outer", 
    left_on="team_key", 
    right_on="team_key"
    )

round_1 = round_1[
    [
        "week", 
        "reg_season_rank",
        "team_key", 
        "final_points", 
        "projected_points"
        ]][
            round_1["team_key"].isin(playoff_teams)
            ]

round_1.sort_values("reg_season_rank", inplace=True)

round_1.drop("reg_season_rank", axis=1, inplace=True)

m = 0
for match in range(1, matches_per_round + 1):
    round_1.loc[
        round_1["team_key"] == round_1_matchups[m][0], "r1_matchup_num"
    ] = f"{playoff_start_week}.{match}"

    round_1.loc[
        round_1["team_key"] == round_1_matchups[m][1], "r1_matchup_num"
    ] = f"{playoff_start_week}.{match}"

    m += 1

round_1 = round_1.merge(
    round_1[["team_key", "r1_matchup_num", "final_points", "projected_points"]],
    how="outer",
    left_on="r1_matchup_num",
    right_on="r1_matchup_num",
    suffixes=("", "_opp"),
)

round_1 = round_1[~round_1["r1_matchup_num"].isna()]

round_1['count'] = round_1.groupby('r1_matchup_num')['team_key'].transform('count')

round_1 = round_1[(round_1['count'] == 1) | (round_1['team_key'] != round_1['team_key_opp'])]

round_1.loc[round_1['team_key_opp'] == round_1['team_key'], 'final_points_opp'] = 0
round_1.loc[round_1['team_key_opp'] == round_1['team_key'], 'projected_points_opp'] = 0
round_1.loc[round_1['team_key_opp'] == round_1['team_key'], 'team_key_opp'] = "Bye"

for row in round_1:
    round_1["win_loss_r1"] = np.where(
        round_1["final_points"] > round_1["final_points_opp"],
        "W",
        "L",
    )

round_1.rename(
    columns={
        "week": "week_r1",
        "final_points": "pts_r1",
        "projected_points": "pro_pts_r1",
        "r1_matchup_num": "matchup_r1",
        "team_key_opp": "opp_team_key_r1",
        "final_points_opp": "opp_pts_r1",
        "projected_points_opp": "opp_pro_pts_r1",
    },
    inplace=True,
)

round_1.drop('count', axis=1, inplace=True)

eos_rankings = eos_rankings.merge(
    round_1,
    how="outer",
    left_on="team_key",
    right_on="team_key",
)

In [3]:
eos_rankings.sort_values(['win_loss_r1', 'matchup_r1'])

,game_id,reg_season_rank,manager,team_name,team_key,week_r1,pts_r1,pro_pts_r1,matchup_r1,opp_team_key_r1,opp_pts_r1,opp_pro_pts_r1,win_loss_r1
7,273,8,James,Taco,273.l.777818.t.1,15.0,65.86,102.73,15.1,273.l.777818.t.7,88.50,106.62,L
1,273,2,273.l.777818.t.10,273.l.777818.t.10,273.l.777818.t.10,15.0,106.34,117.06,15.2,273.l.777818.t.12,NaN,NaN,L
6,273,7,273.l.777818.t.12,273.l.777818.t.12,273.l.777818.t.12,NaN,NaN,NaN,15.2,273.l.777818.t.10,106.34,117.06,L
2,273,3,273.l.777818.t.2,273.l.777818.t.2,273.l.777818.t.2,15.0,71.90,58.54,15.3,273.l.777818.t.4,75.38,85.51,L
4,273,5,273.l.777818.t.9,273.l.777818.t.9,273.l.777818.t.9,15.0,63.68,110.55,15.4,273.l.777818.t.6,77.14,90.89,L
0,273,1,Jake,Goons,273.l.777818.t.7,15.0,88.50,106.62,15.1,273.l.777818.t.1,65.86,102.73,W
5,273,6,Wes,The Fear Boners,273.l.777818.t.4,15.0,75.38,85.51,15.3,273.l.777818.t.2,71.90,58.54,W
3,273,4,Pete,ELE,273.l.777818.t.6,15.0,77.14,90.89,15.4,273.l.777818.t.9,63.68,110.55,W
8,273,9,273.l.777818.t.8,273.l.777818.t.8,273.l.777818.t.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,273,10,Tim,Cudde2,273.l.777818.t.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
